In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import plotly.express as px

os.chdir("../")
from src.conf import LAYERS_DIMS, MODEL_FEATURES
from src.models import FraudAutoEncoder
from src.data import DataLoaders

# Constantes

In [ ]:
SAMPLING_PARAMS = {
    "low": -1, "high": 1, "size" : [5_000, 300]
}

DATA_SPLIT_FRACS = [0.7, 0.2, 0.1]

# Simulated Data for Test

In [ ]:
simul_data = np.random.uniform(**SAMPLING_PARAMS)

_simul_data = torch.from_numpy(simul_data)

INPUT_DIM = _simul_data.shape[1]

# Programs Tests

In [ ]:
# Model configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = FraudAutoEncoder(
    INPUT_DIM,
    LAYERS_DIMS
).to(device)

optimizer = torch.optim.Adam(
    model.parameters(), lr=MODEL_FEATURES.LEARNING_RATE
)

loss_criterion = torch.nn.MSELoss()

In [ ]:
# Get Pytorch Dataloaders
(
    train_dataloader,
    validation_dataloader,
    test_dataloader
) = DataLoaders(_simul_data, DATA_SPLIT_FRACS).get_dataloaders()

In [ ]:
# Model Training
training_losses, validation_losses = [], []

for epoch in range(MODEL_FEATURES.N_EPOCHS):
    for _, inputs in enumerate(train_dataloader):
        # Zero gradients for every batch!
        optimizer.zero_grad()

        # Make encoding and decoding
        reconstitued_inputs = model(inputs)

        # Compute loss and its gradients on trining datasets
        training_loss = loss_criterion(inputs, reconstitued_inputs)
        training_loss.backward()

        # Adjust learning weights
        optimizer.step()

    training_losses.append(float(training_loss))

    # Inference after each epoch on validation data
    for _, validation_inputs in enumerate(validation_dataloader):
        reconstitued_validation_inputs = model(validation_inputs)
        with torch.no_grad():
            validation_loss = loss_criterion(
                validation_inputs,
                reconstitued_validation_inputs
            )

    validation_losses.append(float(validation_loss))

    print("Epoch {:} : Training loss = {:.6f} | Validation loss = {:.6f}".format(epoch, training_loss, validation_loss))

In [ ]:
df_losses = pd.concat(
    [
        pd.DataFrame(
            {
                "epoch": range(MODEL_FEATURES.N_EPOCHS),
                "split": "train",
                "loss": training_losses
            }
        ),
        pd.DataFrame(
            {
                "epoch": range(MODEL_FEATURES.N_EPOCHS),
                "split": "validation",
                "loss": validation_losses
            }
        )
    ]
)

In [ ]:
fig = px.line(
    df_losses,
    x='epoch', y='loss',
    color='split', markers=True,
    height=500, width=800
)
fig.show()